In [1]:
import numpy as np
import pandas as pd
from scipy import stats
from matplotlib import pyplot as plt
import seaborn as sns
import statsmodels.formula.api as smf
import statsmodels.api as sm
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori, association_rules
import warnings
warnings.filterwarnings('ignore')
import missingno
from sklearn.preprocessing import LabelEncoder
import re
from sklearn.metrics import accuracy_score

from matplotlib import font_manager, rc
font_path = "dataset/malgun.ttf"
font_name = font_manager.FontProperties(fname=font_path).get_name()
rc('font', family=font_name)

In [2]:
# 14년과 15년 가을데이터 READ
fall14 = pd.read_pickle('dataset/fall14.pkl')
fall15 = pd.read_pickle('dataset/fall15.pkl')

In [3]:
# 14년 데이터셋에서 고객번호와 고객당 총 구매액만 컬럼화

f14 = fall14.groupby('CUSTNO').sum().reset_index()[['CUSTNO', 'PURCHAMOUNT']]
f14.rename(columns = {'PURCHAMOUNT':'14_purchases'}, inplace = True)

In [4]:
# 15년 데이터셋에서 고객번호와 고객당 총 구매액만 컬럼화

f15 = fall15.groupby('CUSTNO').sum().reset_index()[['CUSTNO', 'PURCHAMOUNT']]
f15.rename(columns = {'PURCHAMOUNT':'15_purchases'}, inplace= True)

In [5]:
# 위의 두 데이터셋을 left join (14년에 구매한 사람은 전부 남게 데이터 프레임 통합)

ori_cust = pd.merge(f14, f15, on='CUSTNO', how='left')
ori_cust

,CUSTNO,14_purchases,15_purchases
0,00001,22124200,6558568.0
1,00002,10952010,14119720.0
2,00003,472176,369408.0
3,00004,2098760,1780825.0
4,00005,270580,1735340.0
...,...,...,...
19236,19374,91630,2096529.0
19237,19375,642649,4821414.0
19238,19376,45510,2372742.0
19239,19377,45500,1889880.0


In [6]:
# spet_less 종속변수를 만들고 고객별로 구매가 감소한 사람은 1, 아닌사람은 0으로 분류
ori_cust['spent_less'] = ori_cust.apply(lambda x: 1 if x['14_purchases']-x['15_purchases']<0
                                        else 0, axis= 1)
ori_cust.head()

,CUSTNO,14_purchases,15_purchases,spent_less
0,00001,22124200,6558568.0,0
1,00002,10952010,14119720.0,1
2,00003,472176,369408.0,0
3,00004,2098760,1780825.0,0
4,00005,270580,1735340.0,1


In [7]:
# 구매감소한 사람과 아닌사람의 숫자 비교
ori_cust.spent_less.value_counts()

1    9808
0    9433
Name: spent_less, dtype: int64

# 재구매율 낮은 제품 판매금액 제거

- 재구매 데이터제거 위해 전처리, 파생변수 생성

In [8]:
# 6가지의 제품군으로 재분류한 결과
# 각각 food(음식), nor(생활용품), cloth(의류), hobby(취미), rich(사치), etc(기타)
# 에 해당하는 코드 (제휴사 + 대분류코드)

total_food = ['A1','B1', 'B2', 'B3', 'B4', 'B5', 'B6',
          'B7', 'B9', 'B10', 'B11', 'B12', 'B13', 'B14', 'B15', 'B16','B37', 'B38',
          'B43', 'B46', 'B47', 'B48','B52', 'B53', 'B54', 'B55',
          'B56', 'B57', 'B58', 'B59', 'B60', 'B61', 'B62', 'B63',
          'B64', 'B65', 'B67', 'B72','B81', 'B82', 'B83', 'B89', 'B91', 'B92',
         'C1', 'C2', 'C3', 'C4', 'C5', 'C6', 'C7', 'C8', 'C9',
         'C10', 'C11', 'C12', 'C13', 'C14', 'C15', 'D4',
        'B73','B74','B75','B76','B77','B78','B79','B80']
# B73에서 B80누락됐던 부분 식품에 포함 

total_nor = ['A2', 'A3', 'A7', 'A9', 'B17',
        'B18', 'B19', 'B20', 'B21', 'B22', 'B23', 'B24', 'B31',
         'B34', 'B66', 'B77', 'B84', 'B86', 'B87', 'B88',
             'C16', 'C17', 'D1', 'D2', 'D3', 'D5', 'D6']

total_cloth = ['A4', 'A5', 'B29','B30', 'B32', 'B33', 'B44',
               'B49', 'B50', 'B51', 'B68', 'B69']

total_hobby = ['A6', 'B25', 'B26', 'B27', 'B35', 'B36', 'B71']

total_rich = ['A8', 'B28', 'B70']
total_etc = ['B8', 'B45', 'B90', 'D7', 'D8']


# 위에서 분류한 것을 기준으로 mapping 진행
def prod_cat(data):
    if data in total_food:
        return 'food'
    elif data in total_nor:
        return 'nor'
    elif data in total_cloth:
        return 'cloth'
    elif data in total_hobby:
        return 'hobby'
    elif data in total_rich:
        return 'luxury'
    elif data in total_etc:
        return 'etc'

In [9]:
# 14,15년 데이터에 'AFFIL_CLASS'와 'prod_cat' column 추가

fall14['CLASSCODEL'] = fall14['CLASSCODEL'].astype(str)
fall15['CLASSCODEL'] = fall15['CLASSCODEL'].astype(str)
fall14['AFFIL_CLASS'] = fall14.apply(lambda x: x['AFFIL'] + x['CLASSCODEL'], axis=1)
fall15['AFFIL_CLASS'] = fall15.apply(lambda x: x['AFFIL'] + x['CLASSCODEL'], axis=1)

# 제휴사(AFFIL) 와 대분류코드(CLASS)를 합친 column인 AFFIL_CLASS 를 바탕으로
# 새로운 제품분류 column인 prod_cat column 생성
fall14['prod_cat'] = fall14.AFFIL_CLASS.map(prod_cat)
fall15['prod_cat'] = fall15.AFFIL_CLASS.map(prod_cat)

- 재구매율 0% 제품 제거

In [10]:
# 재구매율 0%인 제품 제거
fall14 = fall14.drop(fall14.loc[(fall14.AFFIL_CLASS=='B34')|
                                (fall14.AFFIL_CLASS=='B62')|
                               (fall14.AFFIL_CLASS=='B59')|
                               (fall14.AFFIL_CLASS=='B61')|
                               (fall14.AFFIL_CLASS=='B58')|
                               (fall14.AFFIL_CLASS=='B60')|
                               (fall14.AFFIL_CLASS=='B42')|
                               (fall14.AFFIL_CLASS=='B40')|
                               (fall14.AFFIL_CLASS=='B41')|
                               (fall14.AFFIL_CLASS=='B85')].index, axis=0)

fall15 = fall15.drop(fall15.loc[(fall15.AFFIL_CLASS=='B34')|
                                (fall15.AFFIL_CLASS=='B62')|
                               (fall15.AFFIL_CLASS=='B59')|
                               (fall15.AFFIL_CLASS=='B61')|
                               (fall15.AFFIL_CLASS=='B58')|
                               (fall15.AFFIL_CLASS=='B60')|
                               (fall15.AFFIL_CLASS=='B42')|
                               (fall15.AFFIL_CLASS=='B40')|
                               (fall15.AFFIL_CLASS=='B41')|
                               (fall15.AFFIL_CLASS=='B85')].index, axis=0)

# inflation 적용

- inflation 적용 위해 전처리, 파생변수 생성

In [11]:
# 14년 데이터셋에서 고객번호와 고객당 총 구매액만 컬럼화

f14 = fall14.groupby('CUSTNO').sum().reset_index()[['CUSTNO', 'PURCHAMOUNT']]
f14.rename(columns = {'PURCHAMOUNT':'14_purchases'}, inplace = True)
f14.head()

,CUSTNO,14_purchases
0,00001,22124200
1,00002,10952010
2,00003,472176
3,00004,2098760
4,00005,270580


- inflation 적용

In [15]:
# 의류 -0.4%, 취미 1.6% 적용
# 의류와 취미를 제외한 상품군은 5퍼센트로 적용 (일단 차이를 보기위해)

fall15['PURCHAMOUNT_scaled'] = fall15.apply(lambda x: x['PURCHAMOUNT'] * 0.9524 if x['prod_cat']=='food'
                                        else x['PURCHAMOUNT'] * 0.9524 if x['prod_cat']=='nor'
                                        else x['PURCHAMOUNT'] * -0.9960 if x['prod_cat']=='cloth'
                                        else x['PURCHAMOUNT'] * 0.9843 if x['prod_cat']=='hobby'
                                        else x['PURCHAMOUNT'] * 0.9524 if x['prod_cat']=='luxury'
                                        else x['PURCHAMOUNT'] * 0.9524, axis= 1) # etc
fall15['PURCHAMOUNT_scaled'] =  fall15['PURCHAMOUNT_scaled'].astype(float)


In [16]:
# 15년 데이터셋에서 고객번호와 고객당 총 구매액만 컬럼화

f15 = fall15.groupby('CUSTNO').sum().reset_index()[['CUSTNO', 'PURCHAMOUNT_scaled']]
f15.rename(columns = {'PURCHAMOUNT_scaled':'15_purchases_scaled'}, inplace= True)
f15.head()

,CUSTNO,15_purchases_scaled
0,00001,-2.470340e+06
1,00002,1.011036e+06
2,00003,3.518242e+05
3,00004,8.780165e+04
4,00005,1.432471e+06


In [17]:
# 위의 두 데이터셋을 left join (14년에 구매한 사람은 전부 남게 데이터 프레임 통합)

ori_cust = pd.merge(f14, f15, on='CUSTNO', how='left')

# spet_less 종속변수를 만들고 고객별로 구매가 감소한 사람은 1, 아닌사람은 0으로 분류
ori_cust['spent_less'] = ori_cust.apply(lambda x: 1 if x['14_purchases']-x['15_purchases_scaled']<0
                                        else 0, axis= 1)
ori_cust.head()

,CUSTNO,14_purchases,15_purchases_scaled,spent_less
0,00001,22124200,-2.470340e+06,0
1,00002,10952010,1.011036e+06,0
2,00003,472176,3.518242e+05,0
3,00004,2098760,8.780165e+04,0
4,00005,270580,1.432471e+06,1


In [19]:
# 구매감소한 사람과 아닌사람의 숫자 비교
ori_cust.spent_less.value_counts()

0    14529
1     4712
Name: spent_less, dtype: int64

- 인플레이션과 재품군 노이즈 제거함에 따라 구매감소 고객 분류가 달라짐

# 의류와 취미 제품군에서 감소한 고객만 확인

- cloth와 hobby만 포함한 14,15년도 fall Dataframe 생성

In [24]:
cloth_hobby14 = fall14[(fall14.prod_cat == 'cloth')| (fall14.prod_cat =='hobby')]
cloth_hobby15 = fall15[(fall15.prod_cat == 'cloth')| (fall15.prod_cat =='hobby')]


In [25]:
# 14년 데이터셋에서 고객번호와 고객당 총 구매액만 컬럼화

f14 = cloth_hobby14.groupby('CUSTNO').sum().reset_index()[['CUSTNO', 'PURCHAMOUNT']]
f14.rename(columns = {'PURCHAMOUNT':'14_purchases'}, inplace = True)
f14.head()

,CUSTNO,14_purchases
0,00001,4860810
1,00002,6558370
2,00004,1230170
3,00006,606500
4,00007,7204940


In [26]:
# 15년 데이터셋에서 고객번호와 고객당 총 구매액만 컬럼화

f15 = cloth_hobby15.groupby('CUSTNO').sum().reset_index()[['CUSTNO', 'PURCHAMOUNT_scaled']]
f15.rename(columns = {'PURCHAMOUNT_scaled':'15_purchases_scaled'}, inplace= True)
f15.head()

,CUSTNO,15_purchases_scaled
0,00001,-4337260.875
1,00002,-5839720.068
2,00004,-759686.730
3,00005,-112597.800
4,00006,-1737679.500


# 최종 후보.. 내일 질문 후 학습에 6개 상품군 전부 포함할지, cloth와 hobby만 포함할지 결정

In [81]:
# 위의 두 데이터셋을 left join (14년에 구매한 사람은 전부 남게 데이터 프레임 통합)

ori_cust = pd.merge(f14, f15, on='CUSTNO', how='left')

# spet_less 종속변수를 만들고 고객별로 구매가 감소한 사람은 1, 아닌사람은 0으로 분류
ori_cust['spent_less'] = ori_cust.apply(lambda x: 1 if x['14_purchases']-x['15_purchases_scaled']<0
                                        else 0, axis= 1)
ori_cust.head()

,CUSTNO,14_purchases,15_purchases_scaled,spent_less
0,00001,4860810,-4.337261e+06,0
1,00002,6558370,-5.839720e+06,0
2,00004,1230170,-7.596867e+05,0
3,00006,606500,-1.737680e+06,0
4,00007,7204940,-1.279364e+07,0


In [28]:
# 구매감소한 사람과 아닌사람의 숫자 비교
ori_cust.spent_less.value_counts()

0    15945
1      828
Name: spent_less, dtype: int64

- 구매감소 비율이 왜 32.4%에서 5.2%로 낮아졌을까
- 액수

In [68]:
ori_cust[['CUSTNO', 'spent_less']].to_pickle('dataset/label1.pkl')

# 검산

In [63]:
# 제품군을 groupby 하여 묶은 후 2014년의 제품군 별 총 매출액 계산
purch_by_prodcat = fall14.groupby('prod_cat').sum()[['PURCHAMOUNT']]
# 제품군을 groupby 하여 묶은 후 2015년의 제품군 별 총 매출액 계산
purch_by_prodcat15 = fall15.groupby('prod_cat').sum()[['PURCHAMOUNT']]

In [64]:
# 15년의 제품군 별 총액을 _14에 15Purch라는 새로운 column을 생성하여 저장
purch_by_prodcat['15_purchases'] = purch_by_prodcat15.PURCHAMOUNT
purch_by_prodcat.rename(columns = {'PURCHAMOUNT':'14_purchases', }, inplace = True)
purch_by_prodcat

,14_purchases,15_purchases
prod_cat,,
cloth,28269908870,27806480670
etc,982547076,1106168320
food,21067667723,23783732682
hobby,7423925940,7258360920
luxury,7987014700,9536549190
nor,23970841504,24650700801


In [65]:
# 각 제품별 share 확인하는 '15_purch_share(%)' column 생성
purch_by_prodcat.insert(2, '15_purch_share(%)', purch_by_prodcat.apply(lambda x: 
                        round(x['15_purchases']/purch_by_prodcat['15_purchases'].sum()*100,2).sum(), axis=1))

# 14년의 제품군 별 총 금액과 15년의 제품군 별 총 금액의 차이를 계산하여
# diff 라는 새로운 column에 저장
purch_by_prodcat['diff'] = \
purch_by_prodcat.apply(lambda x: round(x['15_purchases'] - x['14_purchases']), axis=1)

# 차이가 얼마나 나왔는지 퍼센트로 계산
purch_by_prodcat['change(%)'] =\
purch_by_prodcat.apply(lambda x: 
                        round(x['diff']/x['15_purchases']*100,2).sum(), axis=1)

purch_by_prodcat

,14_purchases,15_purchases,15_purch_share(%),diff,change(%)
prod_cat,,,,,
cloth,28269908870,27806480670,29.54,-463428200,-1.67
etc,982547076,1106168320,1.17,123621244,11.18
food,21067667723,23783732682,25.26,2716064959,11.42
hobby,7423925940,7258360920,7.71,-165565020,-2.28
luxury,7987014700,9536549190,10.13,1549534490,16.25
nor,23970841504,24650700801,26.18,679859297,2.76


In [72]:
fall1415 = pd.read_pickle('dataset/fall1415.pkl')

In [91]:
# 일단 가을 데이터 전부에서 고객별로 라벨값 추가
full_data_with_label = fall1415.join(ori_cust.set_index('CUSTNO')['spent_less'], on='CUSTNO', how='inner')


In [92]:
full_data_with_label.spent_less.unique()

array([0, 1], dtype=int64)

In [95]:
full_data_with_label.head()

,RECEIPTNO,CUSTNO,PURCHDATE,PURCHTIME,AFFIL,CLASSCODEL,CLASSCODEM,CLASSCODES,STORENO,GENDER,AGEGROUP,RESIDENCE,PURCHAMOUNT,Year,AFFIL_CLASS,spent_less
0,08068631,07122,2014-09-06,17,B,8,0802,B080203,028,M,40세~44세,100,14000,2014,B8,0
35684,08069767,07122,2014-10-16,13,B,18,1802,B180205,028,M,40세~44세,100,29600,2014,B18,0
35685,08069767,07122,2014-10-16,13,B,19,1908,B190801,028,M,40세~44세,100,9900,2014,B19,0
35686,08069767,07122,2014-10-16,13,B,20,2004,B200402,028,M,40세~44세,100,3300,2014,B20,0
39485,08069767,07122,2014-10-16,13,B,38,3803,B380303,028,M,40세~44세,100,1300,2014,B38,0


In [96]:
# 전체 데이터 피클로 저장

full_data_with_label.to_pickle('dataset/full_data_with_label.pkl')